In [90]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
from urllib.request import urlopen
import httplib2
import pandas as pd
import numpy as np

In [91]:
# Set base url for loops and years list
base_url = 'https://www.cs.cmu.edu/~mleone/gdead/'
years = ['72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95']

In [92]:
# Initialize a list to store the txt file paths
concert_list = []

# Loop each year page to get all the txt file paths and add them to the list
for year in years:
    http = httplib2.Http()
    status, response = http.request(base_url+year+'.html')

    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            concert_list.append(link['href'])
    
# print(concert_list)

In [93]:
# Remove the non-file links from the list
del concert_list[1389]
del concert_list[1471:1473]
del concert_list[1471]

In [94]:
# Used to verify that all file paths accounted for and pointing to a txt file (no external links)
print(concert_list)

['dead-sets/72/1-2-72.txt', 'dead-sets/72/3-5-72.txt', 'dead-sets/72/3-21-72.txt', 'dead-sets/72/3-22-72.txt', 'dead-sets/72/3-23-72.txt', 'dead-sets/72/3-25-72.txt', 'dead-sets/72/3-26-72.txt', 'dead-sets/72/3-27-72.txt', 'dead-sets/72/3-28-72.txt', 'dead-sets/72/4-7-72.txt', 'dead-sets/72/4-8-72.txt', 'dead-sets/72/4-11-72.txt', 'dead-sets/72/4-14-72.txt', 'dead-sets/72/4-15-72.txt', 'dead-sets/72/4-16-72.txt', 'dead-sets/72/4-17-72.txt', 'dead-sets/72/4-21-72.txt', 'dead-sets/72/4-24-72.txt', 'dead-sets/72/4-26-72.txt', 'dead-sets/72/4-29-72.txt', 'dead-sets/72/5-3-72.txt', 'dead-sets/72/5-4-72.txt', 'dead-sets/72/5-7-72.txt', 'dead-sets/72/5-10-72.txt', 'dead-sets/72/5-11-72.txt', 'dead-sets/72/5-13-72.txt', 'dead-sets/72/5-16-72.txt', 'dead-sets/72/5-18-72.txt', 'dead-sets/72/5-23-72.txt', 'dead-sets/72/5-24-72.txt', 'dead-sets/72/5-25-72.txt', 'dead-sets/72/5-26-72.txt', 'dead-sets/72/6-17-72.txt', 'dead-sets/72/7-10-72.txt', 'dead-sets/72/7-16-72.txt', 'dead-sets/72/7-21-72.txt'

In [131]:
# Initialize an empty list to add all the concert info to:
concert_setlists = []

In [132]:
for concert in concert_list:
    # Create the url for each txt file then read it into a Soup object
    concert_url = base_url+concert

    with urlopen(concert_url) as link:
        result = link.read()

    soup = BeautifulSoup(result)

    setlist_raw = soup.find('p')
    setlist = setlist_raw.text
    data = setlist.split('\n')

    # Get concert loc and date
    concert_details = data.pop(0)

    # Get the setlist
    songs = list(data)

    # Split the 2-digit year from the concert date
    date_str = concert_details.split()[-1:]
    date = date_str[0]
    year_2dig = date[-3:]
    year_2dig = year_2dig.replace(')', '')

    # Combine the concert details, setlist, and year into a dictionary
    concert_dict = {'Concert':concert_details, 'Setlist':songs, 'Year':'19'+year_2dig}

    # Append the dictionary to the list of concerts/setlists
    concert_setlists.append(concert_dict)


In [133]:
# Verifying that all dictionaries populated as intended
print(concert_setlists)

[{'Concert': 'Winterland Arena, San Francisco, CA (1/2/72)', 'Setlist': ['', "Truckin'", 'Sugaree', 'Mr. Charlie', 'Beat it on Down the Line', 'Loser', 'Jack Straw', 'Chinatown Shuffle', 'Your Love At Home', 'Tennessee Jed', 'El Paso', 'You Win Again', 'Big Railroad Blues', 'Mexicali Blues', 'Playing in the Band', 'Next Time You See Me', 'Brown Eyed Women', 'Casey Jones', '', "Good Lovin'", 'China Cat Sunflower', 'I Know You Rider', "Good Lovin'", 'Ramble On Rose', 'Sugar Magnolia', 'Not Fade Away', "Goin' Down the Road Feeling Bad", 'Not Fade Away', '', 'One More Saturday Night', '', ''], 'Year': '1972'}, {'Concert': 'Winteriand Arena, San Francisco, CA (3/5/72)', 'Setlist': ['', 'Bertha', 'Black Throated Wind', 'Mr. Charlie', 'Sugaree', 'Greatest Story Ever Told', 'Next Time You See Me', 'Tennessee Jed', 'Jack Straw', 'China Cat Sunflower', 'I Know You Rider', 'Mexicali Blues', 'You Win Again', 'El Paso', 'Casey Jones', '', "Good Lovin'", 'Not Fade Away', "Goin' Down the Road Feeling

In [136]:
# Create dataframe from the list of dictionaries
concerts_complete_df = pd.DataFrame.from_dict(concert_setlists)
concerts_complete_df


,Concert,Setlist,Year
0,"Winterland Arena, San Francisco, CA (1/2/72)","[, Truckin', Sugaree, Mr. Charlie, Beat it on ...",1972
1,"Winteriand Arena, San Francisco, CA (3/5/72)","[, Bertha, Black Throated Wind, Mr. Charlie, S...",1972
2,"Academy of Music, New York, NY (3/21/72)","[, [set list unknown], , ]",1972
3,"Academy of Music, New York, NY (3/22/72)","[, [set list unknown], , ]",1972
4,"Academy of Music, New York, NY (3/23/72)","[, China Cat Sunflower, I Know You Rider, Blac...",1972
...,...,...,...
1599,"Deer Creek Amphitheater, Noblesville, IN (Mond...","[, Cancelled on account of gate-crashing 7/2, ]",1995
1600,"Riverport Amphitheatre, Maryland Heights, MO (...","[ , Feel Like A Stranger, Peggy-O, The Same Th...",1995
1601,"Riverport Amphitheatre, Maryland Heights, MO (...","[, Mississippi Half Step, Take me to the River...",1995
1602,"Soldier Field, Chicago, IL (Saturday 7/8/95)","[, Jack Straw, Sugaree, Wang Dang Doodle, Alth...",1995


In [137]:
#JSON Normalize - to split the song lists out to rows:
setlists_all_df = pd.json_normalize(concert_setlists, record_path='Setlist', meta=['Concert', 'Year'])
setlists_all_df

,0,Concert,Year
0,,"Winterland Arena, San Francisco, CA (1/2/72)",1972
1,Truckin',"Winterland Arena, San Francisco, CA (1/2/72)",1972
2,Sugaree,"Winterland Arena, San Francisco, CA (1/2/72)",1972
3,Mr. Charlie,"Winterland Arena, San Francisco, CA (1/2/72)",1972
4,Beat it on Down the Line,"Winterland Arena, San Francisco, CA (1/2/72)",1972
...,...,...,...
39635,Box of Rain,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39636,,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39637,* Weir on acoustic,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39638,*(The Band opened)*,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995


In [138]:
# Clean up empty songs rows and extra punctuation, rename 'Song' column
setlists_all_df[0].replace('', np.nan, inplace=True)
setlists_all_df.dropna(subset=[0], inplace=True)
setlists_all_df[0] = setlists_all_df[0].str.replace('->','')
setlists_all_df.rename({0:'Song'}, axis=1, inplace=True)

setlists_all_df

,Song,Concert,Year
1,Truckin',"Winterland Arena, San Francisco, CA (1/2/72)",1972
2,Sugaree,"Winterland Arena, San Francisco, CA (1/2/72)",1972
3,Mr. Charlie,"Winterland Arena, San Francisco, CA (1/2/72)",1972
4,Beat it on Down the Line,"Winterland Arena, San Francisco, CA (1/2/72)",1972
5,Loser,"Winterland Arena, San Francisco, CA (1/2/72)",1972
...,...,...,...
39632,Sugar Magnolia,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39634,E: Black Muddy River,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39635,Box of Rain,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995
39637,* Weir on acoustic,"Soldier Field, Chicago, IL (Sunday 7/9/95)",1995


In [139]:
# Write the dataframe to a csv
setlists_all_df.to_csv('setlists_72to95.csv', index=False)